In [1]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers.pooling import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, UpSampling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
import os
import numpy as np
from keras.applications.imagenet_utils import preprocess_input 
from keras.utils import np_utils
import random
import pprint
import cPickle
from scipy.misc import imread, imresize, imsave
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.engine.topology import Merge, merge
import matplotlib.pyplot as plt
from PIL import Image
from keras.utils.visualize_util import plot

Using TensorFlow backend.


ImportError: No module named pydot

In [ ]:
%matplotlib inline

In [ ]:
train_datagen = ImageDataGenerator(
        featurewise_center=True,
        rescale=1./255,
        dim_ordering='tf')

In [ ]:
random.seed(0)

X_train = []
Y_train = []

for directory in os.listdir('./data/'):
    l = os.listdir('./data/' + directory)
    for image in l:
        image = l[0]
        img = load_img('./data/' + directory + '/' + image, target_size=(224,224))  # this is a PIL image
        img = img.convert("YCbCr")
        x = img_to_array(img)
        Y_train.append(x[:,:,1:])
        X_train.append(x[:,:,:1])
    break
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [ ]:
XX_train = np.zeros((X_train.shape[0], X_train.shape[1], X_train.shape[2], X_train.shape[3] + 2))
XX_train[:,:,:,0:] = X_train
XX_train.shape

In [ ]:
def getModel():
    ''' 
        * return: compiled model (keras.engine.training.Model)
    '''
    vgg_model = VGG16( weights='imagenet', include_top=True )
    for i in range(9):
        vgg_model.layers.pop()    
    
    bnl = BatchNormalization() (vgg_model.layers[-1].output)
    cnl = Convolution2D(256, 1, 1, border_mode='same', dim_ordering='tf') (bnl)
    usl = UpSampling2D(dim_ordering='tf') (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-5].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(128, 3, 3, border_mode='same', dim_ordering='tf') (ml)

    usl = UpSampling2D(dim_ordering='tf') (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-9].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(64, 3, 3, border_mode='same', dim_ordering='tf') (ml)

    usl = UpSampling2D(dim_ordering='tf') (cnl)
    bnl = BatchNormalization() (vgg_model.layers[-12].output)
    ml = merge([usl, bnl], mode='sum')
    cnl = Convolution2D(3, 3, 3, border_mode='same', dim_ordering='tf') (ml)

    bnl = BatchNormalization() (vgg_model.layers[-14].output)
    ml = merge([cnl, bnl], mode='sum')
    cnl = Convolution2D(3, 3, 3, border_mode='same', dim_ordering='tf') (ml)

    cnl = Convolution2D(2, 3, 3, border_mode='same', dim_ordering='tf') (cnl)

    tl_model = Model( input=vgg_model.input, output=cnl )
    for i in range(1, 14):
        tl_model.layers[i].trainable = False

    return tl_model

In [ ]:
##loss function

# def blur_uv_loss(rgb, inferred_rgb):
#     uv = rgb2uv(rgb)
#     uv_blur0 = rgb2uv(blur(rgb, 3))
#     uv_blur1 = rgb2uv(blur(rgb, 5))

#     inferred_uv = rgb2uv(inferred_rgb)
#     inferred_uv_blur0 = rgb2uv(blur(inferred_rgb, 3))
#     inferred_uv_blur1 = rgb2uv(blur(inferred_rgb, 5))

#     return  ( dist(inferred_uv, uv) +
#             dist(inferred_uv_blur0 , uv_blur0) +
#             dist(inferred_uv_blur1, uv_blur1) ) / 3

In [ ]:
tl_model = getModel()
# pprint.PrettyPrinter(indent=2).pprint(tl_model.get_config())
tl_model.summary()

tl_model.compile(loss="mse", optimizer="adam", metrics=["acc"])

In [ ]:
train_datagen.fit(X_train)

In [ ]:
# history = tl_model.fit_generator(train_datagen.flow(X_train,Y_train,batch_size=88), samples_per_epoch=88, nb_epoch=50, validation_data=(X_valid,Y_valid), verbose=1)

history = tl_model.fit(XX_train[:10,:,:,:],Y_train[:10,:,:,:],batch_size=5, nb_epoch=100, verbose=1, shuffle=True)

# tl_model.save(filepath='./output/urban_2_images')
# cPickle.dump( history.history, open( "urbnan_2_images.p", "wb" ) )

In [ ]:
a = tl_model.predict(XX_train[0:1, :, :, :])

In [ ]:
a = a.reshape(224,224,2)

In [ ]:
i = np.append(XX_train[0,:,:,0:1], a, axis=2)

In [ ]:
b = Image.fromarray(i, mode='YCbCr').convert('RGB')

In [ ]:
c = img_to_array(b)

In [ ]:
plt.imshow(c)